**Mongo DB Notebook**

In [ ]:

# docker network create my-mongo-cluster

# docker run --name mongo1 -d --net my-mongo-cluster -p 9042:9042 mongo --replSet "rs0" --port 9042
# docker run --name mongo2 -d --net my-mongo-cluster -p 9142:9142 mongo --replSet "rs0" --port 9142
# docker run --name mongo3 -d --net my-mongo-cluster -p 9242:9242 mongo --replSet "rs0" --port 9242
    
    
# docker exec -it mongo1 mongo --port 9042
# config = {
#     "_id" : "rs0", 
#     "members" : [
#         {"_id" : 0,"host" : "mongo1:9042"},
#         {"_id" : 1,"host" : "mongo2:9142"},
#         {"_id" : 2,"host" : "mongo3:9242"}]
# }
# rs.initiate(config)
# rs.status() 



In [65]:
import pymongo
import pprint
import datetime
from bson.json_util import dumps
from bson.objectid import ObjectId
from bson.dbref import DBRef


client = pymongo.MongoClient(host=['localhost:9042'], replicaSet='rs0')
db = client["distributed-db"]

In [66]:
# list could be used 
def print_cursor(cursor):
    for doc in cursor:
        pprint.pprint(doc)

In [67]:
# Продемонструвати запис даних на primary node з різними Write Concern Levels 

# Unacknowledged
write_concern = pymongo.write_concern.WriteConcern(w=0)

replicas = db.replicas.with_options(write_concern=write_concern)

insert_one_result = replicas.insert_one({"unacknowledged": 1})

print(write_concern.document)
print(insert_one_result.acknowledged)

{'w': 0}
False


In [68]:
# Acknowledged
write_concern = pymongo.write_concern.WriteConcern(w=1)

replicas = db.replicas.with_options(write_concern=write_concern)

insert_one_result = replicas.insert_one({"acknowledged": 1})

print(write_concern.document)
print(insert_one_result.acknowledged)

{'w': 1}
True


In [69]:
# Journaled

write_concern = pymongo.write_concern.WriteConcern(w=1, j=True)

replicas = db.replicas.with_options(write_concern=write_concern)

insert_one_result = replicas.insert_one({"journaled": 1})

print(write_concern.document)
print(insert_one_result.acknowledged)

{'j': True, 'w': 1}
True


In [70]:
# AcknowledgedReplica

write_concern = pymongo.write_concern.WriteConcern(w=3)

replicas = db.replicas.with_options(write_concern=write_concern)

insert_one_result = replicas.insert_one({"AcknowledgedReplica": 1})

print(write_concern.document)
print(insert_one_result.acknowledged)

{'w': 3}
True


In [71]:
# Продемонструвати Read Preference Modes: читання з 
# primary

print(client.read_preference)

print_cursor(db.replicas.find({}))

Primary()
{'_id': ObjectId('60698134e006941af7c2d813'), 'unacknowledged': 1}
{'_id': ObjectId('60698176e006941af7c2d814'), 'acknowledged': 1}
{'_id': ObjectId('60698178e006941af7c2d815'), 'journaled': 1}
{'AcknowledgedReplica': 1, '_id': ObjectId('6069817ce006941af7c2d816')}


In [72]:
# secondary node 
secondary_client = pymongo.MongoClient(host=['localhost:9042'], replicaSet='rs0', readPreference='secondary')
secondary_db = secondary_client["distributed-db"]
secondary_client.read_preference

print_cursor(secondary_db.replicas.find({}))

{'_id': ObjectId('60698134e006941af7c2d813'), 'unacknowledged': 1}
{'_id': ObjectId('60698176e006941af7c2d814'), 'acknowledged': 1}
{'_id': ObjectId('60698178e006941af7c2d815'), 'journaled': 1}
{'AcknowledgedReplica': 1, '_id': ObjectId('6069817ce006941af7c2d816')}


In [86]:
!docker container ls

CONTAINER ID   IMAGE     COMMAND                  CREATED          STATUS          PORTS                               NAMES
010dd273632f   mongo     "docker-entrypoint.s…"   50 minutes ago   Up 3 minutes    0.0.0.0:9242->9242/tcp, 27017/tcp   mongo3
0ac55ae6a20d   mongo     "docker-entrypoint.s…"   50 minutes ago   Up 50 minutes   0.0.0.0:9142->9142/tcp, 27017/tcp   mongo2
048c2ec7b8e9   mongo     "docker-entrypoint.s…"   55 minutes ago   Up 55 minutes   0.0.0.0:9042->9042/tcp, 27017/tcp   mongo1


In [87]:
# Спробувати зробити запис з однією відключеною нодою та write concern рівнім 3 та нескінченім таймаутом. 
# Спробувати під час таймаута включити відключену ноду 
!docker stop mongo3

mongo3


In [74]:

write_concern = pymongo.write_concern.WriteConcern(w=3, wtimeout=None)

replicas = db.replicas.with_options(write_concern=write_concern)

insert_one_result = replicas.insert_one({"WithTimeout": 1})

print(write_concern.document)
print(insert_one_result.acknowledged)


{'w': 3}
True


In [88]:
!docker container start mongo3

mongo3


In [89]:
# Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. 
# Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”
!docker stop mongo3

mongo3


In [80]:
# Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. 
# Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”

write_concern = pymongo.write_concern.WriteConcern(w=3, wtimeout=5000)
read_concern = pymongo.read_concern.ReadConcern(level='majority')

replicas = db.replicas.with_options(write_concern=write_concern, read_concern=read_concern)

try:
    insert_one_result = replicas.insert_one({"WithTimeout": 5000})
except pymongo.errors.WTimeoutError:
    print('timeout')

print(write_concern.document)

print(insert_one_result.acknowledged)

timeout
{'wtimeout': 5000, 'w': 3}
True


In [90]:
!docker container start mongo3

mongo3


In [83]:
# Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”
print(read_concern.level)

print_cursor(replicas.find({"WithTimeout": 5000}))

majority
{'WithTimeout': 5000, '_id': ObjectId('60698bd9e006941af7c2d81d')}


In [93]:
# Продемонстрував перевибори primary node в відключивши поточний primary (Replica Set Elections) - 
client.primary

('mongo1', 9042)

In [101]:
!docker stop mongo1

mongo1


In [102]:
insert_one_result = db.replicas.insert_one({"NewPrimary": 2})
print(insert_one_result.acknowledged)

True


In [103]:
# і що після відновлення роботи старої primary на неї реплікуються нові дані, які з'явилися під час її простою
!docker container start mongo1

print(client.primary)

read_concern = pymongo.read_concern.ReadConcern(level='local')

replicas = db.replicas.with_options(read_concern=read_concern)

print_cursor(replicas.find({"NewPrimary": 2}))

mongo1
('mongo2', 9142)
{'NewPrimary': 2, '_id': ObjectId('6069900fe006941af7c2d820')}


In [117]:
client.primary

('mongo2', 9142)

In [ ]:
# Привести кластер до неконсистентного стану користуючись моментом часу коли 
# primary node не відразу помічає відсутність secondary node 

In [118]:
!docker stop mongo2

mongo2


In [119]:
client.primary

('mongo1', 9042)

In [120]:
!docker stop mongo3

import time 
time.sleep(2)

# відключити останню secondary node та протягом 5 сек. на мастері записати значення (з w:1) і перевірити, що воно записалось

client = pymongo.MongoClient(host=['localhost:9042'], replicaSet='rs0')
db = client["distributed-db"]

write_concern = pymongo.write_concern.WriteConcern(w=1)

replicas = db.replicas.with_options(write_concern=write_concern)
insert_one_result = replicas.insert_one({"IncosistentPrimary": 1})

print_cursor(replicas.find({"IncosistentPrimary": 1}))

mongo3
{'IncosistentPrimary': 1, '_id': ObjectId('6069bb8fe006941af7c2d824')}


In [123]:
# спробувати зчитати це значення з різними рівнями read concern

client = pymongo.MongoClient(host=['localhost:9042'], replicaSet='rs0', readPreference='secondaryPreferred')
db = client["distributed-db"]

read_concern = pymongo.read_concern.ReadConcern(level='majority')
replicas = db.replicas.with_options(read_concern=read_concern)

try:
    print_cursor(db.replicas.find({"IncosistentPrimary": 1}))
except pymongo.errors.ServerSelectionTimeoutError:
    print('Server Selection Error') 

{'IncosistentPrimary': 1, '_id': ObjectId('6069bb8fe006941af7c2d824')}


In [124]:
read_concern = pymongo.read_concern.ReadConcern(level='local')
replicas = db.replicas.with_options(read_concern=read_concern)

try:
    print_cursor(db.replicas.find({"IncosistentPrimary": 1}))
except pymongo.errors.ServerSelectionTimeoutError:
    print('Server Selection Error') 

{'IncosistentPrimary': 1, '_id': ObjectId('6069bb8fe006941af7c2d824')}


In [125]:
read_concern = pymongo.read_concern.ReadConcern(level='lineralizable')
replicas = db.replicas.with_options(read_concern=read_concern)

try:
    print_cursor(db.replicas.find({"IncosistentPrimary": 1}))
except pymongo.errors.ServerSelectionTimeoutError:
    print('Server Selection Error') 

{'IncosistentPrimary': 1, '_id': ObjectId('6069bb8fe006941af7c2d824')}


In [126]:
# включити дві інші ноди таким чином, 
# щоб вони не бачили попереднього мастера (його можна відключити) і дочекатись поки вони оберуть нового мастера 
!docker stop mongo1

mongo1


In [127]:
!docker container start mongo2
!docker container start mongo3

mongo2
mongo3


In [137]:
client = pymongo.MongoClient(host=['localhost:9142'], replicaSet='rs0')
db = client["distributed-db"]

In [138]:
client.primary

('mongo3', 9242)

In [140]:
# підключити (включити) попередню primary-ноду до кластеру і подивитись, що сталось зі значенням яке було на неї записано
!docker container start mongo1

print_cursor(db.replicas.find({"IncosistentPrimary": 1}))

mongo1


In [155]:
# Земулювати eventual consistency за допомогою установки затримки реплікації для репліки 
!docker exec -it mongo1 mongo --port 9042 --eval "printjson(rs.conf())"

MongoDB shell version v4.4.4
connecting to: mongodb://127.0.0.1:9242/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("9e1daad0-1cdf-469f-850a-cdab900f819e") }
MongoDB server version: 4.4.4
{
	"_id" : "rs0",
	"version" : 1,
	"term" : 5,
	"protocolVersion" : NumberLong(1),
	"writeConcernMajorityJournalDefault" : true,
	"members" : [
		{
			"_id" : 0,
			"host" : "mongo1:9042",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : false,
			"priority" : 1,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(0),
			"votes" : 1
		},
		{
			"_id" : 1,
			"host" : "mongo2:9142",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : false,
			"priority" : 1,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(0),
			"votes" : 1
		},
		{
			"_id" : 2,
			"host" : "mongo3:9242",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : false,
			"priority" : 1,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(0),
		

In [166]:
!docker exec -it mongo3 mongo --port 9242 --eval "cfg = rs.conf(); cfg.members[0].priority = 0; cfg.members[0].hidden = true; cfg.members[0].slaveDelay = 5; rs.reconfig(cfg)"

MongoDB shell version v4.4.4
connecting to: mongodb://127.0.0.1:9242/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("b1c7d993-15e6-4de8-8e6c-90aa71ec417f") }
MongoDB server version: 4.4.4
{
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1617543867, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1617543867, 1)
}


In [167]:
!docker exec -it mongo1 mongo --port 9042 --eval "printjson(rs.conf())"

MongoDB shell version v4.4.4
connecting to: mongodb://127.0.0.1:9042/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("4b68e84f-1301-4714-bd29-fe0f3c1170d0") }
MongoDB server version: 4.4.4
{
	"_id" : "rs0",
	"version" : 3,
	"term" : 5,
	"protocolVersion" : NumberLong(1),
	"writeConcernMajorityJournalDefault" : true,
	"members" : [
		{
			"_id" : 0,
			"host" : "mongo1:9042",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : true,
			"priority" : 0,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(5),
			"votes" : 1
		},
		{
			"_id" : 1,
			"host" : "mongo2:9142",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : false,
			"priority" : 1,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(0),
			"votes" : 1
		},
		{
			"_id" : 2,
			"host" : "mongo3:9242",
			"arbiterOnly" : false,
			"buildIndexes" : true,
			"hidden" : false,
			"priority" : 1,
			"tags" : {
				
			},
			"slaveDelay" : NumberLong(0),
			

In [162]:
!docker stop mongo2

mongo2


In [163]:
replicas.insert_one({"WithDelay": 1})
replicas.insert_one({"WithDelay": 2})

In [168]:
read_concern = pymongo.read_concern.ReadConcern(level='linearizable')

replicas = db.replicas.with_options(read_concern=read_concern)

print_cursor(replicas.find({"WithDelay": 1}))
print_cursor(replicas.find({"WithDelay": 2}))

{'WithDelay': 1, '_id': ObjectId('6069c1c2e006941af7c2d82d')}
{'WithDelay': 2, '_id': ObjectId('6069c1c2e006941af7c2d82e')}
